In [1]:
from transformers import XLMForSequenceClassification

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from transformers import AutoTokenizer

In [ ]:
model = XLMForSequenceClassification.from_pretrained('xlm-roberta-base', num_labels=6000, 
    problem_type="multi_label_classification")

In [5]:
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')

In [8]:
model.cuda().eval()
inputs = tokenizer("Generate taxonomy for query: dildo", return_tensors="pt", 
    padding='max_length', truncation=True, max_length=50).to('cuda')

In [12]:
%%timeit
model(**inputs)

12.4 ms ± 73.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [13]:
import torch._dynamo as torchdynamo
torchdynamo.config.cache_size_limit = 512

In [14]:
model.forward2 = torchdynamo.optimize("inductor")(model.forward)

In [15]:
model.forward2(**inputs)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

SequenceClassifierOutput(loss=None, logits=tensor([[-0.9991,  0.0562, -0.2497,  ...,  0.4295, -0.6082, -0.4930]],
       device='cuda:0', grad_fn=<CompiledFunctionBackward>), hidden_states=None, attentions=None)

In [16]:
%%timeit
model.forward2(**inputs)

5.81 ms ± 14.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [17]:
inputs2 = tokenizer("Generate taxonomy for query: xyz iasd adasd qwedqwdq dqwedqw", return_tensors="pt", 
    padding='max_length', truncation=True, max_length=50).to('cuda')

In [18]:
%%timeit
model.forward2(**inputs2)

5.85 ms ± 25.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
